## <center>Лабораторная работа № 8 'Генерация текста на основе “Алисы в стране чудес”'<center>

### <center>Выполнила студентка 3 курса группы БФИ2001 Калмыкова Дарья<center>

### Цель
Использовать рекуррентные нейронные сети в качестве генеративных моделей.

### Задачи
* Ознакомиться с генерацией текста
* Ознакомиться с системой Callback в Keras

### Требования
1. Реализовать модель ИНС, которая будет генерировать текст
2. Написать собственный CallBack, который будет показывать то как генерируется 
текст во время обучения (то есть раз в какое-то количество эпох генирировать и 
выводить текст у необученной модели)
3. Отследить процесс обучения при помощи TensorFlowCallBack (TensorBoard), в 
отчете привести результаты и их анализ

In [1]:
import numpy
import codecs
import re
import sys

from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [2]:
fileObj = codecs.open( "./wonderland.txt", "r", "utf_8" )
raw_text = fileObj.read()
text_clear = re.sub(r"[\r\n]", '', raw_text)
raw_text = text_clear.lower()

In [3]:
fileObj.close()

In [26]:
# raw_text

In [4]:
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [6]:
n_chars = len(raw_text)
print("Total Characters: ", n_chars)

Total Characters:  141208


In [7]:
n_vocab = len(chars)
print("Total Vocab: ", n_vocab)

Total Vocab:  48


In [8]:
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  141108


In [9]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [10]:
model = Sequential()

model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

In [11]:
# define the checkpoint
filepath = "weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', 
                             verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [12]:
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

Epoch 1/20
1103/1103 [==============================] - ETA: 0s - loss: 2.9994
Epoch 1: loss improved from inf to 2.99945, saving model to weights-improvement-01-2.9994.hdf5
1103/1103 [==============================] - 340s 306ms/step - loss: 2.9994
Epoch 2/20
1103/1103 [==============================] - ETA: 0s - loss: 2.8329
Epoch 2: loss improved from 2.99945 to 2.83286, saving model to weights-improvement-02-2.8329.hdf5
1103/1103 [==============================] - 337s 306ms/step - loss: 2.8329
Epoch 3/20
1103/1103 [==============================] - ETA: 0s - loss: 2.7443
Epoch 3: loss improved from 2.83286 to 2.74433, saving model to weights-improvement-03-2.7443.hdf5
1103/1103 [==============================] - 359s 325ms/step - loss: 2.7443
Epoch 4/20
1103/1103 [==============================] - ETA: 0s - loss: 2.6822
Epoch 4: loss improved from 2.74433 to 2.68223, saving model to weights-improvement-04-2.6822.hdf5
1103/1103 [==============================] - 366s 332ms/step - l

In [15]:
# load the network weights
filename = "weights-improvement-20-2.0652.hdf5"
model.load_weights(filename)

model.compile(loss='categorical_crossentropy', optimizer='adam')

In [16]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [18]:
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")

# generate characters
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]

print("\nDone.")

Seed:
" re was no more to be said.at last the mouse, who seemed to be a person of authority among them,calle "
 her head  and the garter wothd the gar and the was soe kant of the care and the was sorednlng to to tea it tat  the was soenk on the tonle th the woudd of the care and the was so tork to toeke th the woudd of the doure tf the woudd of the dareeni,ana the gadt was soe kante was anl toene to the woudd of the dareenii the care an                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                